In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.0 MB/s eta 0:00:0000:0100:01m


In [2]:
import requests
from urllib.parse import urlencode


def download(public_key: str, file_name: str):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его
    download_response = requests.get(download_url)
    with open(file_name, 'wb') as f:   # Здесь укажите нужный путь к файлу
        f.write(download_response.content)


In [3]:
 download('https://disk.yandex.ru/d/Hu-um0ASI6eAUg', 'dataset.zip')

In [ ]:
!unzip -q dataset.zip

In [17]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel/tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

0_CLIPModel/special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

0_CLIPModel/vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

0_CLIPModel/config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

0_CLIPModel/preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

0_CLIPModel/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [40]:
matrix = model.encode([Image.open(f'dataset/{i:0>4}.png') for i in range(9605)])

Batches:   0%|          | 0/301 [00:00<?, ?it/s]

In [41]:
similarities = model.similarity(matrix, matrix)

In [47]:
import torch

In [53]:
value, indices = torch.topk(similarities, 7, largest=False)

In [75]:
res = []

for i in range(9605):
    mas = indices[i].tolist()
    mas.remove(i)
    key = f'{i:0>4}.png'
    value = ' '.join(f'{val:0>4}.png' for val in mas)
    res.append((key, value))

In [78]:
submission = pd.DataFrame(res, columns=['filename', 'ranking'])

In [79]:
submission.head()

,filename,ranking
0,0000.png,5924.png 2023.png 8222.png 1381.png 2750.png 8...
1,0001.png,5029.png 5193.png 5721.png 3598.png 3986.png 7...
2,0002.png,3977.png 2448.png 1858.png 2193.png 7688.png 2...
3,0003.png,7490.png 9555.png 4541.png 7425.png 5277.png 8...
4,0004.png,0258.png 6371.png 7238.png 2961.png 4593.png 2...


In [80]:
submission.to_csv('submission.csv', sep=',')

In [12]:
!ls dataset -l . | egrep -c '^-'

9606


In [13]:
import clip

ModuleNotFoundError: No module named 'clip'